In [1]:
'''
New Jersey Forest Service 2022

This program takes .dat inputs generated by the
csv_to_dat converter and runs them in a linear 
optimization model using pyomo and the glpk
solver.

Code adapted from "Named Generic Abstract Solver"
demonstrated by Michael Gorbunov 2022;
Allows sets' indicies to be named
'''

'\nNew Jersey Forest Service 2022\n\nThis program takes .dat inputs generated by the\ncsv_to_dat converter and runs them in a linear \noptimization model using pyomo and the glpk\nsolver.\n\nCode adapted from "Named Generic Abstract Solver"\ndemonstrated by Michael Gorbunov 2022;\nAllows sets\' indicies to be named\n'

In [2]:
#install Pyomo and glpk solver in Colab environment
%%capture
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

In [3]:
import sys
import pyomo.environ as pyo

In [4]:
#Colab mount Google Drive and map paths
from google.colab import drive
drive.mount('gdrive')
#Google drive paths
COLABPATH = 'gdrive/My Drive/Colab Notebooks/'
DATAPATH = 'gdrive/My Drive/Colab Notebooks/data/'
#add COLABPATH to system path to load classes
#and custom modules
sys.path.insert(0, COLABPATH)

#Path for input file
filepath = DATAPATH+'SLmonthly_1.dat'

Mounted at gdrive


In [5]:
# In Python2, integer divisions truncate values (1/2 = 0 instead of 0.5)
# which breaks the solver. Either way, we should be using python3
if (sys.version[0] != '3'):
	print("You are not using python 3, so this will fail")
	sys.exit(1)

In [6]:
# Below we instantiate the model
model = pyo.AbstractModel()

In [7]:
model.index_vars = pyo.Set()
model.index_le_consts = pyo.Set()
model.index_ge_consts = pyo.Set()
model.index_eq_consts = pyo.Set()

model.x = pyo.Var(model.index_vars, domain=pyo.NonNegativeReals)

In [8]:
# These guys are read from input.dat
model.vec_objective = pyo.Param(model.index_vars)

model.mat_le = pyo.Param(model.index_le_consts, model.index_vars)
model.vec_le = pyo.Param(model.index_le_consts)

model.mat_ge = pyo.Param(model.index_ge_consts, model.index_vars)
model.vec_ge = pyo.Param(model.index_ge_consts)

model.mat_eq = pyo.Param(model.index_eq_consts, model.index_vars)
model.vec_eq = pyo.Param(model.index_eq_consts)

In [9]:
# Defining the actual functions
def obj_function(_model):
	return pyo.summation(_model.vec_objective, _model.x)

def le_mat_rule (_model, k):
	return sum(_model.mat_le[k, i] * _model.x[i] for i in _model.index_vars) <= _model.vec_le[k]

def ge_mat_rule (_model, k):
	return sum(_model.mat_ge[k, i] * _model.x[i] for i in _model.index_vars) >= _model.vec_ge[k]

def eq_mat_rule (_model, k):
	return sum(_model.mat_eq[k, i] * _model.x[i] for i in _model.index_vars) == _model.vec_eq[k]

model.OBJ = pyo.Objective(rule=obj_function, sense=pyo.maximize)
model.GEConstraint = pyo.Constraint(model.index_ge_consts, rule=ge_mat_rule)
model.LEConstraint = pyo.Constraint(model.index_le_consts, rule=le_mat_rule)
model.EQConstraint = pyo.Constraint(model.index_eq_consts, rule=eq_mat_rule)

In [10]:
# Now read data file
instance = model.create_instance(filename=filepath)
solved_model = pyo.SolverFactory('glpk')

In [11]:
# Now optimize
results = solved_model.solve(instance)

In [12]:
#print outputs
'''
This part of the program could use improvements;
process some of this information into outputfiles
or databases
'''
print()
print(instance.OBJ.pprint())
print(instance.EQConstraint.pprint())
print(instance.GEConstraint.pprint())
print(instance.LEConstraint.pprint())

print('\n\n==== Nicer Formatting ====')

for v in instance.component_objects(pyo.Objective, active=True):
	print("Objective Value: ", pyo.value(v))

print()
print("Decision Variables")
for v in instance.component_objects(pyo.Var, active=True):
	for index in v:
		print(index, " ", pyo.value(v[index]))


OBJ : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 40.0*x[a1] + 50.0*x[a2] - x[dummy0] - x[dummy1]
None
EQConstraint : Size=1, Index=index_eq_consts, Active=True
    Key      : Lower : Body                  : Upper : Active
    dummyEQ0 :   0.0 : x[dummy0] + x[dummy1] :   0.0 :   True
None
GEConstraint : Size=0, Index=index_ge_consts, Active=True
    Key : Lower : Body : Upper : Active
None
LEConstraint : Size=3, Index=index_le_consts, Active=True
    Key     : Lower : Body                  : Upper   : Active
    Acreage :  -Inf :         x[a1] + x[a2] :  1600.0 :   True
     Budget :  -Inf : 4.0*x[a1] + 8.0*x[a2] : 10000.0 :   True
      Ccred :  -Inf :                 x[a1] :  1000.0 :   True
None


==== Nicer Formatting ====
Objective Value:  73000.0

Decision Variables
a1   700.0
a2   900.0
dummy0   0.0
dummy1   0.0
